# **Training and Evaluation with build-in methods**

## **Setup && Introduction**

In [1]:
import tensorboard

import tensorflow as tf
import tensorflow.keras as tfk
from tensorflow.keras import layers as lyrs

import numpy as np
import matplotlib.pyplot as plt

In [2]:
# HYPERPARAMETERS uniform through out the code
BS = 64 # Batch_Size
EP = 10 # Epochs
VB = 1 # Verbose

## **API overview: An End-to-End Example**

In [3]:
# Creating a basic Mnist Classifier for starting
(tr_data, tr_lbls), (ts_data, ts_lbls) = tfk.datasets.mnist.load_data()

tr_data = tr_data.reshape(-1, 28*28).astype('float32') / 255.0
ts_data = ts_data.reshape(-1, 28*28).astype('float32') / 255.0

val_data = tr_data[:-10000] # Reserve last 10,000 images for validation set
val_lbls = tr_lbls[:-10000] # Take last 10,000 labels as-well.

In [5]:
# Define the Model Structure and Compile
Model_mnist = tfk.Sequential(
    [
        lyrs.Dense(128, activation='relu', input_shape=(784,), name= 'InputLayer'),
        lyrs.Dense(64, activation='relu', name= 'Dense_1'),
        lyrs.Dense(10, activation='softmax', name= 'OutputLayer'),
    ], name= 'Digits_Recognition_Model'
)

Model_mnist.summary()

Model_mnist.compile(
    optimizer=tfk.optimizers.Nadam(),
    loss= tfk.losses.SparseCategoricalCrossentropy(),
    metrics= [tfk.metrics.SparseCategoricalAccuracy()]
)

Model: "Digits_Recognition_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (Dense)          (None, 128)               100480    
                                                                 
 Dense_1 (Dense)             (None, 64)                8256      
                                                                 
 OutputLayer (Dense)         (None, 10)                650       
                                                                 
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Training the Model and seeing the result!!
Model_mnist.fit(
    tr_data, tr_lbls,
    validation_data= (val_data, val_lbls),
    epochs= EP, verbose= VB,
    batch_size= BS
)

Epoch 1/10
938/938 [==============================] - 9s 8ms/step - loss: 0.2718 - sparse_categorical_accuracy: 0.9227 - val_loss: 0.1264 - val_sparse_categorical_accuracy: 0.9626
Epoch 2/10
938/938 [==============================] - 7s 7ms/step - loss: 0.1131 - sparse_categorical_accuracy: 0.9657 - val_loss: 0.0867 - val_sparse_categorical_accuracy: 0.9737
Epoch 3/10
938/938 [==============================] - 7s 7ms/step - loss: 0.0754 - sparse_categorical_accuracy: 0.9771 - val_loss: 0.0650 - val_sparse_categorical_accuracy: 0.9802
Epoch 4/10
938/938 [==============================] - 7s 7ms/step - loss: 0.0574 - sparse_categorical_accuracy: 0.9822 - val_loss: 0.0427 - val_sparse_categorical_accuracy: 0.9873
Epoch 5/10
938/938 [==============================] - 7s 7ms/step - loss: 0.0446 - sparse_categorical_accuracy: 0.9862 - val_loss: 0.0311 - val_sparse_categorical_accuracy: 0.9907
Epoch 6/10
938/938 [==============================] - 7s 7ms/step - loss: 0.0349 - sparse_categorica

In [7]:
# Evaluating the Model on Test Data!!
Model_mnist.evaluate(ts_data, ts_lbls, verbose= VB, batch_size= BS)

157/157 [==============================] - 1s 4ms/step - loss: 0.0906 - sparse_categorical_accuracy: 0.9763A: 0s - loss: 0.0977 - sparse_categorical_accuracy: 0.9


[0.09056906402111053, 0.9763000011444092]

## **The Compile Method**

In [8]:
# A vauge Compile Function
Model_mnist.compile(
    optimizer= 'Adam',
    loss= 'MSE',
    metrics= ['accuracy']
)
# Typically, the compile function takes 3 main arguments
# OPTIMIZER, LOSS_FUNCTION,  and EVALUATION_METRIC(optional)

def get_model():
    Inp = tfk.Input(shape= (784, ))
    lyr = lyrs.Dense(64, activation='relu')(Inp)
    lyr = lyrs.Dense(64, activation='relu')(lyr)
    Out = lyrs.Dense(10)(lyr)

    return tfk.Model(inputs= Inp, outputs= Out)

def compile_model(model):
    model.compile(
        optimizer= 'Adam',
        loss= 'MSE',
        metrics= ['accuracy']
    )
    return model

# Once the Model is defined, it can be compiled
compiled_model = compile_model(get_model())

### **Built-in optimizers, losses and metrics**

**Visit `https://keras.io/api/` for in-depth information on inbuilt losses, metrics etc**

### **Custom Losses**
> **2 Methods of creating custom losses**
1. **Functional approach -- pass y_true and y_pred to a func and compute the difference between the two values**
2. **Sub-classing approach -- Implement methods __init__() and call()**

In [11]:
# Functional Approach
def Mean_SQ_Error(y_true, y_pred): 
    return tf.math.reduce_mean(tf.square(y_true - y_pred))


# Getting a Vauge Model calling a previously created function
model = get_model()

# Defining prams for compile
metric = tfk.metrics.SparseCategoricalAccuracy()
opt = tfk.optimizers.Adamax()
loss = Mean_SQ_Error

model.compile(
    optimizer= tfk.optimizers.Adamax(0.001),
    loss= Mean_SQ_Error,
    # metrics= [metric]
)

OneHot_lbls = tf.one_hot(tr_lbls, depth= 10)

# model.fit(tr_data, OneHot_lbls, epochs= EP, batch_size= BS)
model.fit(tr_data, OneHot_lbls, batch_size=64, epochs=5)

# Sparse Catagorical Cross Entropy will not work here...
...

# Custom Loss not working so-far

Epoch 1/5
938/938 [==============================] - 4s 3ms/step - loss: 0.0315A: 3s -  - ETA: 
Epoch 2/5
938/938 [==============================] - 3s 4ms/step - loss: 0.0160
Epoch 3/5
938/938 [==============================] - 3s 3ms/step - loss: 0.0124
Epoch 4/5
938/938 [==============================] - 3s 3ms/step - loss: 0.0104
Epoch 5/5
938/938 [==============================] - 3s 3ms/step - loss: 0.0091


Ellipsis

In [14]:
# Sub-Classing using tf.keras.losses.Loss class
class MeSqEr_Custom(tfk.losses.Loss):
    def init(self, regfac= 0.1):
        super().__init__()
        self.regularization_factor = regfac
    
    def call(self, ytrue, ypred):
        SqErr1 = tf.square(ytrue - ypred)
        mse = tf.reduce_mean(SqErr1)
        
        SqErr2 = tf.square(0.5 - ypred)
        reg = tf.reduce_mean(SqErr2)
        
        cstm_loss = mse + reg * self.regularization_factor
        return cstm_loss
    

class CustomMSE(tfk.losses.Loss):
    def __init__(self, regularization_factor=0.1, name="custom_mse"):
        super().__init__(name=name)
        self.regularization_factor = regularization_factor

    def call(self, y_true, y_pred):
        mse = tf.math.reduce_mean(tf.square(y_true - y_pred))
        reg = tf.math.reduce_mean(tf.square(0.5 - y_pred))
        return mse + reg * self.regularization_factor

# Get a Model    
model = get_model()

# Define prams
# optimizer and metric already defined from the last block: opt, metric
# cstm_mse = MeSqEr_Custom(0.25)
cstm_mse = CustomMSE()

model.compile(
    optimizer= opt,
    loss= cstm_mse,
    # metrics= [metric]
)

tr_lbls_OneHot = tf.one_hot(tr_lbls, depth= 10)
model.fit(tr_data, tr_lbls_OneHot, epochs=EP, verbose=VB, batch_size=BS)

# Custom Loss not working so-far

Epoch 1/10
938/938 [==============================] - 4s 3ms/step - loss: 0.0562
Epoch 2/10
938/938 [==============================] - 3s 4ms/step - loss: 0.0396
Epoch 3/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0356
Epoch 4/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0334
Epoch 5/10
938/938 [==============================] - 2s 3ms/step - loss: 0.0319
Epoch 6/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0308
Epoch 7/10
938/938 [==============================] - 2s 3ms/step - loss: 0.0300A: 0s - los
Epoch 8/10
938/938 [==============================] - 2s 3ms/step - loss: 0.0294
Epoch 9/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0290
Epoch 10/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0286


### **Custom Metrics**

In [23]:
'''
    To implement custom metrics, you need to subclass the 
    tf.metrics.Metric class and implement 4 functions
    1. init()   2. update_state()   3. reset_state()    4. result()  
'''

# Here's a simple example showing how to implement a CategoricalTruePositives metric 
# that counts how many samples were correctly classified as belonging to a given class

class cstm_metric(tf.metrics.Metric):
    def __init__(self): #  create state variables for your metric
        super(cstm_metric, self).__init__()
        self.true_positives = self.add_weight(
            name= 'ctp',
            initializer= 'zeros'
        )
        
    # uses the targets y_true and the model predictions y_pred to update the state variables.
    def update_state(self, y_true, y_pred, sample_weight= None):
        y_pred = tf.reshape(
            tf.argmax(y_pred, axis= 1), shape=(-1, 1)
        )
        
        values = tf.cast(y_true, 'int32') == tf.cast(y_pred, 'int32')
        values = tf.cast(values, 'float32')
        
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, 'float32')
            values = values * sample_weight
            
        self.true_positives.assign_add(tf.reduce_sum(values))
    
    def result(self):  # Uses the state variables to compute the final results.
        return self.true_positives
    
    def reset_state(self): # Reinitializes the state of the metric.
        self.true_positives.assign_add(0.0)

In [24]:
# Testing the new Metric
model = get_model()

model.compile(
    optimizer=opt,
    loss= tfk.losses.SparseCategoricalCrossentropy(),
    metrics= [cstm_metric()]
)

model.fit(tr_data, tr_lbls, epochs= EP)

Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 2.0930 - cstm_metric_4: 39203.0000A: 1s - loss: 2.2932 -
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 1.0989 - cstm_metric_4: 87185.0000
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.9718 - cstm_metric_4: 138115.0000
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 1.1591 - cstm_metric_4: 189174.0000
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 1.4413 - cstm_metric_4: 239906.0000
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 1.4619 - cstm_metric_4: 290638.0000
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 1.6659 - cstm_metric_4: 339026.0000
Epoch 8/10
1875/1875 [==============================] - 5s 3ms/step - loss: 1.8644 - cstm_metric_4: 384502.0000
Epoch 9/10
1875/1875 [==============================] - 5s 3ms/step - loss: 1.8580 -

### **Handling losses and metrics that donot fit the standard signature**

> **To achieve this, we call the add_loss() and add_metric() methods from within the call function of a custom Layer, and the metric and/or loss gets added to the main loss/metric passed to the model at the compile time**

#### **Handling Loss**

In [26]:
print("Dealing loss here!!")
class ActReg(lyrs.Layer):
    def call(self, inputs):
        # Using add_los mothod here to alter loss computation...
        self.add_loss(tf.reduce_sum(inputs) * 0.1)
        return inputs # Just Pass through the Layer

inp = tfk.Input(shape= (784, ))
x_ten = lyrs.Dense(64, 'relu')(inp)
x_ten = ActReg()(x_ten)

x_ten = lyrs.Dense(64, 'relu')(x_ten)
out = lyrs.Dense(10, 'relu')(x_ten)

model = tfk.Model(inputs= inp, outputs= out)

model.compile(
    opt, 
    loss= tfk.losses.SparseCategoricalCrossentropy(from_logits= True)
)

model.fit(tr_data, tr_lbls, epochs= EP, verbose= VB, batch_size= BS)

Dealing loss here!!
Epoch 1/10
938/938 [==============================] - 5s 4ms/step - loss: 3.1213
Epoch 2/10
938/938 [==============================] - 4s 4ms/step - loss: 2.3093
Epoch 3/10
938/938 [==============================] - 3s 4ms/step - loss: 2.3040
Epoch 4/10
938/938 [==============================] - 4s 4ms/step - loss: 2.3025
Epoch 5/10
938/938 [==============================] - 3s 4ms/step - loss: 2.3020
Epoch 6/10
938/938 [==============================] - 3s 4ms/step - loss: 2.3018
Epoch 7/10
938/938 [==============================] - 3s 4ms/step - loss: 2.3018
Epoch 8/10
938/938 [==============================] - 3s 4ms/step - loss: 2.3017
Epoch 9/10
938/938 [==============================] - 4s 4ms/step - loss: 2.3017
Epoch 10/10
938/938 [==============================] - 4s 4ms/step - loss: 2.3017


#### **Handling Metrcis**

In [ ]:
print("Dealing metrics here!!")
class MetricLoggingLayer(tfk.layers.Layer):
    #
    #
    def __init__(self):
        super(MetricLoggingLayer, self).__init__()
        
        

### **Automatically setting apart a validation holdout set**

## **Training and Evaluation from tf.Datasets**


### **Using a Validation dataset**

## **Other supported input formats**

## **Using tfk.utils.Sequence object as input**

## **Using sample weighting and class weighting**

### **Class Weights**

### **Sample Weights**

## **Passing data to multi-input/output models**

## **Using Callbacks**

### **Built-in callbacks**


### **Writing custom callbacks**


## **Model Checkpoints**

## **Usign learning-rate schedules**

### **Passing a schedule to an optimizer**

### **Using callbacks to implement a dynamic learning rate schedule**

## **Visualizing loss and metrics during training**

### **Using Tensorboard callback**

# **Wrapup**